# DataSport Scraping

In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bfs
from IPython.display import HTML
import pandas as pd
import numpy as np
import time
import random
from astropy.io import ascii

## Scraping a table

In [ ]:
HTML('<iframe src=https://services.datasport.com/2015/lauf/transviamala/alfaw.htm width=1000 height=350></iframe>')

In [ ]:
def get_probability_of_split(table, column):
    nb_blank = 0
    for line in table:
        if len(line) > column and line[column] == ' ':
            nb_blank += 1
    return nb_blank / len(table)

def split_separator(probabilities, separator):
    max_prob = max(probabilities, key=lambda x: x['probability'])
    index = max_prob['index']
    separator = separator[:index] + ' ' + separator[index + 1:]
    return separator

def read_table(content):
    header = content[0]
    separator = content[1]
    table = content[2:]
    
    blank = False
    probabilities = []
    
    for column in range(0, len(header)):
        if header[column] == ' ':
            blank = True
            new_prob = {'index': 0, 'probability': 0}
            new_prob['index'] = column
            new_prob['probability'] = get_probability_of_split(table, column)
            probabilities.append(new_prob)
            side = 'right'
        elif blank and header[column] != ' ':
            blank = False
            separator = split_separator(probabilities, separator)
            probabilities = []

    final_content = [header, separator] + table

    return ascii.read(final_content, format='fixed_width_two_line')

In [ ]:
page = rq.get('https://services.datasport.com/2015/lauf/transviamala/alfac.htm')
soup = bfs(page.text, 'html5lib')

content = get_content(soup)

read_table(content)

In [ ]:
page = rq.get('https://services.datasport.com/2009/diverse/trophy/alfac.htm')
soup = bfs(page.text, 'html5lib')

content = get_content(soup)

# TODO fix ...
#read_table(content)

## Scraping a run

In [ ]:
HTML('<iframe src=https://services.datasport.com/2015/lauf/transviamala width=1000 height=350></iframe>')

In [ ]:
def get_all_data_from_page(url):
    data = []
    page = rq.get(url)
    print('status request: ' + str(page.status_code))
    soup = bfs(page.text, 'html5lib')
    
    table_linksCap = soup.select('font > a[href*=ALF]')
    # Elements in 1999 have Alf taf instead ALF
    table_linksNor = soup.select('font > a[href*=Alf]')
    
    if table_linksCap > table_linksNor:
        table_links =  table_linksCap
    else:
        table_links =  table_linksNor
    
    for idx, link in enumerate(table_links):
        full_link = url + link['href']
        
        print(str(idx+1) + '/' + str(len(table_links)) + ' - Processing ' + full_link)
        
        alpha_page = rq.get(full_link)
        alpha_bfs = bfs(alpha_page.text, 'html5lib')
        df = create_clean_dataframe(get_raw_table(alpha_bfs).text, get_header(alpha_bfs).text)
        data.append(df)
        
        time.sleep(random.uniform(0, 0.5))
    return data

In [ ]:
#data = get_all_data_from_page('https://services.datasport.com/1999/lauf/sion/')

### Get data from all run event

In [ ]:
run_events_df = pd.read_csv('Data/run_events.csv')
run_events_df.head()

In [ ]:
def get_data_from_run_events(run_events, path='Data/ScrappingAcodeByRuns/'):

    nbRaceProcessing = 0;
    for run_event in run_events.itertuples():
        print('Processing: Run "' + run_event.name + '" / Date ' + run_event.full_date + ' / Url ' + run_event.url)
        print('Processing the url: ' + run_event.url)
        data_run_events = get_all_data_from_page( run_event.url + '/')
        
        # Merge the data.
        result_race_event = pd.concat(data_run_events)
        print(len(result_race_event))
            
        # write on csv.
        result_race_event.to_csv(path + run_event.name + '.csv')
        
        # test 10 first url.
        nbRaceProcessing += 1
        if nbRaceProcessing > 10:
            break


In [ ]:
#get_data_from_run_events(run_events_df)

> URL qui pose probleme, à regarder de plus pres.

In [ ]:
#data_run_event = get_all_data_from_page('http://services.datasport.com/2009/diverse/trophy/')

In [ ]:
#data = get_all_data_from_page('http://services.datasport.com/1999/lauf/Greifenseelauf/')